In [1]:
import pickle

import numpy as np
import tensorflow as tf
# from keras.applications import InceptionV3
from efficientnet.model import EfficientNetB7
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Flatten
from keras_preprocessing.image.image_data_generator import ImageDataGenerator

In [2]:
with open('../data/raw/train_data.pkl', 'rb') as f:
    x_train = pickle.load(f)

with open('../data/raw/train_labels.pkl', 'rb') as f:
    y_train = pickle.load(f)


In [3]:
data = zip(x_train, y_train)

In [4]:
i = 0
data_4 = []
for x, y in data:
    if y == 4:
        i += 1
        if i > 700:
            continue
    data_4.append((x, y))

i = 0
data_11 = []
for x, y in data_4:
    if y == 11:
        i += 1
        if i > 700:
            continue
    data_11.append((x, y))

x_train, y_train = zip(*data_11)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
datagen.fit(x_train)

In [5]:
y_train = to_categorical(y_train)

In [6]:
base = EfficientNetB7(
    input_shape=(224, 224, 3), 
    include_top=False, 
    weights='imagenet',
    backend=tf.keras.backend,
    layers=tf.keras.layers,
    models=tf.keras.models,
    utils=tf.keras.utils
)
base.trainable = False

model = models.Sequential()
model.add(base)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(14, activation='softmax'))

In [10]:
model = InceptionV3(
    input_shape=(224, 224, 3), 
    include_top=False, 
    weights='imagenet'
)
x = model.output

In [11]:
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(14, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)

AttributeError: 'Node' object has no attribute 'output_masks'

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
x_train = np.asarray(x_train)

In [9]:
x_train = x_train / 255.

In [24]:
x_train.shape

(8857, 224, 224, 3)

In [10]:
model.fit(x_train, y_train, epochs=4)

Train on 8857 samples
Epoch 1/4
8857/8857 [==============================] - 113s 13ms/sample - loss: 5.4435 - accuracy: 0.3321
Epoch 2/4
8857/8857 [==============================] - 99s 11ms/sample - loss: 1.9943 - accuracy: 0.4425
Epoch 3/4
8857/8857 [==============================] - 98s 11ms/sample - loss: 1.5878 - accuracy: 0.5056
Epoch 4/4
8857/8857 [==============================] - 103s 12ms/sample - loss: 1.5888 - accuracy: 0.5333


In [11]:
model.fit(x_train, y_train, epochs=1)

Train on 8857 samples
8857/8857 [==============================] - 104s 12ms/sample - loss: 1.3519 - accuracy: 0.5562


In [32]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=2)

Epoch 1/2
277/276 [==============================] - 109s 394ms/step - loss: 2.0513 - accuracy: 0.3153
Epoch 2/2
277/276 [==============================] - 110s 396ms/step - loss: 2.0308 - accuracy: 0.3248


In [20]:
model.save('model')
model.save_weights('model_weights')

In [12]:
with open('../data/raw/test_data.pkl', 'rb') as f:
    x_test = np.asarray(pickle.load(f))

In [14]:
y_pred = model.predict(x_test / 255.)

In [15]:
y_test = np.argmax(y_pred, axis=1)

In [16]:
from collections import Counter

c = Counter(y_test)

In [17]:
c

Counter({11: 411,
         13: 170,
         4: 542,
         8: 219,
         10: 782,
         2: 97,
         5: 348,
         6: 193,
         12: 236,
         0: 285,
         1: 418,
         7: 285,
         9: 253,
         3: 233})

In [68]:
Counter(y_train)

Counter({4: 700,
         3: 561,
         6: 455,
         9: 547,
         2: 458,
         8: 667,
         11: 700,
         5: 581,
         10: 755,
         1: 863,
         12: 815,
         0: 603,
         13: 547,
         7: 605})

In [18]:
import pandas as pd

In [20]:
pd.DataFrame(y_test, columns=['Category']).to_csv("test_submission.csv")